In [1]:
with open('input') as f:
    intcode = [int(n) for n in f.read().split(',')]

In [2]:
def process_intcode(intcode, grid):
    intcode = {i: instruction for i, instruction in enumerate(intcode)}
    pos = (0, 0)
    grid[pos] = '.'
    next_pos = None
    relative_base = 0
    i = 0
    while True:
        initial = str(intcode.get(i, 0)).rjust(5, '0')
        a, b, c, *opcode = initial
        opcode = int(''.join(opcode))
        assert opcode in (1, 2, 3, 4, 5, 6, 7, 8, 9), opcode

        if c == '0':
            p1 = intcode.get(i + 1, 0)
        elif c == '1':
            p1 = i + 1 
        elif c == '2':
            p1 = intcode.get(i + 1, 0) + relative_base

        if b == '0':
            p2 = intcode.get(i + 2, 0)
        elif b == '1':
            p2 = i + 2
        elif b == '2':
            p2 = intcode.get(i + 2, 0) + relative_base

        if a == '0':
            p3 = intcode.get(i + 3, 0)
        elif a == '1':
            p3 = i + 3
        elif a == '2':
            p3 = intcode.get(i + 3, 0) + relative_base

        if opcode == 1:
            intcode[p3] = intcode.get(p1, 0) + intcode.get(p2, 0)
            i += 4
        elif opcode == 2:
            intcode[p3] = intcode.get(p1, 0) * intcode.get(p2, 0)
            i += 4
        elif opcode == 3:
            next_pos, direction_code = get_move(pos, grid)
            intcode[p1] = direction_code
            i += 2
        elif opcode == 4:
            output_value = intcode.get(p1, 0)
            if output_value == 0:
                grid[next_pos] = '#'
            elif output_value == 1:
                pos = next_pos
                grid[pos] = '.'
                next_pos = None
            elif output_value == 2:
                pos = next_pos
                grid[pos] = 'X'
                return grid, pos
            i += 2
        elif opcode == 5:
            if intcode.get(p1, 0) != 0:
                i = intcode.get(p2, 0)
            else:
                i += 3
        elif opcode == 6:
            if intcode.get(p1, 0) == 0:
                i = intcode.get(p2, 0)
            else:
                i += 3
        elif opcode == 7:
            intcode[p3] = 1 if intcode.get(p1, 0) < intcode.get(p2, 0) else 0
            i += 4
        elif opcode == 8:
            intcode[p3] = 1 if intcode.get(p1, 0) == intcode.get(p2, 0) else 0
            i += 4
        elif opcode == 9:
            relative_base += intcode.get(p1, 0)
            i += 2

In [3]:
import random

def get_move(pos, grid):
    x, y = pos
    possible_moves = {
        (x, y - 1): 1,
        (x, y + 1): 2,
        (x - 1, y): 3,
        (x + 1, y): 4,
    }
    walls = {p for p, t in grid.items() if t == '#'}
    possible_moves_not_walls = set(possible_moves) - walls
    next_pos = random.choice(list(possible_moves_not_walls))
    direction_code = possible_moves[next_pos]
    return next_pos, direction_code

In [4]:
def display_grid(grid):
    min_x = min(p[0] for p in grid)
    max_x = max(p[0] for p in grid)
    min_y = min(p[1] for p in grid)
    max_y = max(p[1] for p in grid)

    for y in range(min_y, max_y + 1):
        for x in range(min_x, max_x + 1):
            if (x, y) == (0, 0):
                print('S', end='')
            else:
                print(grid.get((x, y), '?'), end='')
        print()

In [5]:
grid = {}
for i in range(5):
    grid, pos = process_intcode(intcode, grid)
    print()
    display_grid(grid)


?????????????????????????????###?????????
????????????????????????????#...#????????
????????????????????????????#.#.#????????
????????????????????????????#.#.#????????
???????????????????????????##.#.##???????
??????????????????????????#...#...#??????
??????????????????????????#.#####.#??###?
??????????????????????????#.#...#.#?#...#
??????????????????????????#.###.#.###.#.#
??????????????????????????#.#...#.....#.#
??????????????????????????#.#.#########.#
??????????????????????????#.#.#...#.....#
???????????????????????####.#.#.#.#.####?
??????????????????????#...#.#...#.#.....#
?????????#######?###?##.#.#.#.###.#####.#
????????#.......#...#S..#...#.#?#.....#.#
????????#.#####.#.#.#########.#??####.#.#
????????#.....#...#.......#...#?????#...#
?????????####.###########.#.##???????###?
?????.??#.....#.......#...#.#????????????
???##.#?#.#####.#####.#.###.#????????????
??#...#?#...#.#.....#.#.#...#????????????
?##.##???##.#.#.###.#.#.#.##?????.#??###?
#...#?????#.#.#...#.#.#...#??????

In [6]:
min(grid), max(grid)

((-21, -20), (19, 18))

In [7]:
def shortest_route(grid, start, end, visited):
    this_visited = visited | {start}
    if start == end:
        return len(visited)
    x, y = start
    all_possible_moves = {(x, y - 1), (x, y + 1), (x - 1, y), (x + 1, y)}
    walls = {p for p, t in grid.items() if t == '#'}
    possible_moves = set(all_possible_moves) - walls - this_visited
    if len(possible_moves) == 0:
        return None
    if len(possible_moves) == 1:
        start = possible_moves.pop()
        return shortest_route(grid, start, end, this_visited)
    possible_routes = []
    for start in possible_moves:
        num_moves = shortest_route(grid, start, end, this_visited)
        if num_moves is not None:
            possible_routes.append(num_moves)
    if len(possible_routes) > 0:
        return min(possible_routes)

In [8]:
print("Part 1:")
print(shortest_route(grid, start=(0, 0), end=pos, visited=set()))

Part 1:
262


In [9]:
def fill_room_with_oxygen(grid, pos):
    spaces = {p for p, t in grid.items() if t == '.'}
    walls = {p for p, t in grid.items() if t == '#'}
    oxygenated_spaces = {pos}
    i = 1
    last_round = {pos}
    while True:
        i += 1
        this_round = set()
        for x, y in last_round:
            this_round |= {(x, y - 1), (x, y + 1), (x - 1, y), (x + 1, y)} - walls - oxygenated_spaces
        oxygenated_spaces |= this_round
        last_round = this_round
        if len(oxygenated_spaces) == len(spaces):
            return i

In [10]:
print("Part 2:")
print(fill_room_with_oxygen(grid, pos))

Part 2:
314
